# Introduction to Portfolio Optimization with Python

$\textbf{by Ahmed Pirzada, University of Bristol}$

$\textbf{aj.pirzada@bristol.ac.uk}$

$\textbf{26th November 2025}$

# Learning Objectives

- Understand the inputs to a portfolio problem: assets, period, prices, and returns.
- Compute daily and annualised returns and the covariance matrix.
- Formulate and solve mean–variance optimisation with constraints (weights sum to 1, no shorting).
- Interpret optimal weights, risk (variance), and expected return; plot results.
- Find the tangency portfolio by maximising the Sharpe ratio and compare allocations.


## Step 1: Import libraries we need

- Install and Import `yfinance` library to download stock price data from Yahoo Finance

In [1]:
# Install yfinance to download stock price data from Yahoo Finance.
!pip install yfinance


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\herbd\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Import yfinance library to the environment
import yfinance as yf

- Import `numpy` library to handle numerical operations

In [3]:
import numpy as np  # To handle numerical operations

- Import `minimize` function from `scipy` library for optimisation

In [4]:
from scipy.optimize import minimize  # For solving optimization problems

- Import `matplotlib.pyplot` for visualisation

In [5]:
import matplotlib.pyplot as plt  # For plotting charts
import plotly.express as px  # For interactive visualizations

## Step 2: Specify which stocks we want to analyse and define the time period

- Choose tickers and the sample window; this controls which data are downloaded and analysed.

In [11]:
tickers = ['NVDA', 'GOOGL', 'TSLA', 'MSFT']   # List of stocks
start_date = '2015-01-01'
end_date = '2025-10-01'

## Step 3: Download adjusted close prices.


- Download adjusted close price data from Yahoo Finance

In [10]:
# Get Data: yf.download(List, start= , end= , auto_adjust=True)['Close']
data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)['Close']

Failed to get ticker 'MSFT' reason: Expecting value: line 1 column 1 (char 0)
Failed to get ticker 'GOOGL' reason: Expecting value: line 1 column 1 (char 0)
Failed to get ticker 'NVDA' reason: Expecting value: line 1 column 1 (char 0)
[                       0%                       ]Failed to get ticker 'TSLA' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  4 of 4 completed

4 Failed downloads:
['MSFT', 'GOOGL', 'NVDA', 'TSLA']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


- Use `head()` method to preview your data

In [8]:
# # Display the first few rows of the data
data.head()

Ticker,GOOGL,MSFT,NVDA,TSLA
Date,,,,


## Step 4: Visualize prices for these stocks

- Use `plotly` library to visually inspect data using `px.line()`

In [9]:
# Plotly line chart: px.line(Data, title= , labels={ , })
fig = px.line(data,
              title='stock prices',
              labels={"index": "Date", "value": "Price"}
)

fig.update_layout(template='plotly_white',
                  title_x=0.5)

fig.show()

## Step 5: Calculate daily returns from prices

- Use `pct_change()` method from `numpy` library to compute daily percentage returns for each of your stock.
- Use `dropna()` method to drop missing observations.

In [10]:
# Daily returns calculation
returns = data.pct_change().dropna()  # Drop the first row which will be NaN

## Step 6: Estimate expected annual returns and covariance matrix for returns

- Use `mean()` method to calculate average (expected) stock returns and annualise these.

In [11]:
# Multiplying daily values by 252 converts them approximately to annual values, assuming 252 trading days per year
expected_returns = returns.mean() * 252


- Use `cov()` method to calculate variance-covariance matrix for stock returns.

In [12]:
# Calculate the covariance matrix of returns
cov_matrix = returns.cov() * 252

- Print expected returns and the covariance matrix

In [13]:
# Print expected returns and covariance matrix

print(f"\nExpected annual returns:")
print(expected_returns)

print("\nCovariance matrix:")
print(cov_matrix)


Expected annual returns:
Ticker
GOOGL    0.248981
MSFT     0.275725
NVDA     0.674961
TSLA     0.486111
dtype: float64

Covariance matrix:
Ticker     GOOGL      MSFT      NVDA      TSLA
Ticker                                        
GOOGL   0.083032  0.054248  0.074193  0.063690
MSFT    0.054248  0.073162  0.080152  0.063731
NVDA    0.074193  0.080152  0.239567  0.117014
TSLA    0.063690  0.063731  0.117014  0.336222


## Step 7: VISUALISE data for Expected Returns and Covaraince Matrix

- $\textbf{Expected Returns}$: Use `plotly` library to plot the bar chart using `px.bar()`

In [14]:
# Ploty bar chart: fig = px.bar(Data, x= , y= , color= , title= , labels= )
fig = px.bar(
    expected_returns,
    x=expected_returns.index,
    y=expected_returns.values,
    color=expected_returns.index,
    title="Expected Returns",
    labels={"x": "Asset", "y": "Expected Return"}
)

fig.update_layout(
    template="plotly_white",
    title_x=0.5,             # centre title
    showlegend=False
)

fig.show()

- $\textbf{Covariance Matrix}$: Use `plotly` library to plot the heatmap using `px.imshow()`

In [15]:
# Ploty heatmap: fig = px.imshow(Data, title= , labels= , text_auto=True , color_continuous_scale="Reds" , aspect="auto" )
fig = px.imshow(
    cov_matrix,
    title="Covariance Matrix Heatmap",
    labels=dict(x="", y=""),
    text_auto=True,          # shows numbers in each cell
    color_continuous_scale="Reds",  # Shades of red color
    aspect="auto"
)


fig.update_layout(
    template="plotly_white",
    title_x=0.5
)

fig.show()

## Step 7: Setup the Optimisation Problem i.e. Objective and Constraints

- Define the objective (Sharpe Ratio), constraints (weights sum=1, bounds) and the initial guess

In [16]:
# Define Objective Function: Maximising the Shape Ratio is the same as Minimising the -ve of the Sharpe Ratio
def neg_sharpe_ratio(weights, expected_returns, cov_matrix):

    portfolio_return = np.dot(weights, expected_returns)

    portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    return - portfolio_return / portfolio_risk

# Constraints: weights sum to 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

# No short selling: weights between 0 and 1
bounds = tuple((0,1) for _ in tickers)

# Initial guess: equal distribution
init_guess = np.array(len(tickers) * [1. / len(tickers)])

## Step 8: Run the optimization using Sequential Least Squares Programming (SLSQP)

- Use the `minimize` function from `scipi` library to solve the optimisation problem

In [18]:
# Minimise function: miximize(Objective, Initial Guess, args=(Expected Returns, Covariance Matrix), method='SLSQP', bounds= , constraints= )
result = minimize(neg_sharpe_ratio, init_guess, args=(expected_returns, cov_matrix),
                  method='SLSQP', bounds=bounds, constraints=constraints)

result

     message: Optimization terminated successfully
     success: True
      status: 0
         fun: -1.4209232281515798
           x: [ 1.673e-02  2.150e-01  6.344e-01  1.339e-01]
         nit: 7
         jac: [ 1.527e-04 -6.112e-04  3.076e-04 -5.082e-04]
        nfev: 36
        njev: 7
 multipliers: [ 1.234e-04]

## Step 9: VISUALISE optimal allocation in a bar chart and Calculate portfolio risk, return, sharpe ratio

- Get information on optimal weights from the results

In [19]:
# Get weights which maximise the Sharpe Ratio
sharpe_weights = result.x

- Visualise Optimal Weights using Bar Chart

In [20]:
# Plotly bar chart: fig = px.bar(Data, x= , y= , color= , title= , labels= )
fig = px.bar(sharpe_weights,
            x=tickers,
            y=sharpe_weights,
            color=tickers,
            title="Optimal Portfolio Weights (Max Sharpe Ratio)",
            labels={"x": "Asset", "y": "Weight"})

fig.update_layout(
    template="plotly_white",
    title_x=0.5,             # centre title
    showlegend=False
)

fig.show()

$\textbf{Calculate:}$
- $\textbf{Portfolio return}$: $R^{*}$ = $\textbf{w}^{T}\mu$
- $\textbf{Portfolio risk}$: $\sigma^{*}$ = $\sqrt{\textbf{w}^{T} \Omega \textbf{w}}$
- $\textbf{Sharpe ratio}$: $S^{*}$ = $\frac{R^{*}}{\sigma^{*}}$

- Use `np.dot()` method to calculate portfolio return using sharpe_weights and expected_returns

In [21]:
# Portfolio return: R* = w^T * mu
sharpe_return = np.dot(sharpe_weights, expected_returns)

- Use `sqrt()` and `np.dot()` to calculate portfolio risk using sharpe_weights and expected_returns

In [22]:

# Portfolio risk: σ* = sqrt(w^T * Ω * w)
# Hint: you need two dot products i.e. np.dot( , np.dot( , ) )
sharpe_risk = np.sqrt(np.dot(sharpe_weights.T, np.dot(cov_matrix, sharpe_weights)))


- Use `f"..."` to print the results

In [23]:

print(f"\nOptimal Portfolio Expected Return: {sharpe_return:.2%}")
print(f"\nOptimal Portfolio Risk (Std Dev): {sharpe_risk:.2%}")
print(f"\nOptimal Portfolio Sharpe Ratio: {sharpe_return/sharpe_risk:.2f}")



Optimal Portfolio Expected Return: 55.67%

Optimal Portfolio Risk (Std Dev): 39.18%

Optimal Portfolio Sharpe Ratio: 1.42


## Step 10: Plot the efficient frontier

- Generate many random portfolios to trace the efficient frontier of risk vs return.

In [ ]:
# Generate a range of target returns to explore
num_portfolios = 10000
results = np.zeros((3, num_portfolios))
num_assets = len(tickers)

for i in range(num_portfolios):
    # Generate random weights that sum to 1
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    # Calculate portfolio return and risk
    port_return = np.dot(weights, expected_returns)
    port_risk = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    # Store results
    results[0,i] = port_risk
    results[1,i] = port_return
    results[2,i] = (port_return) / port_risk  # Sharpe ratio (risk-free rate not subtracted)


# Plotly scatter plot: px.scatter(x= , y= , c= , cmap= , marker= , s= , alpha= )
fig = px.scatter(
    x=results[0,:],
    y=results[1,:],
    color=results[2,:],
    color_continuous_scale='Viridis',
    title='Efficient Frontier',
    labels={'x': 'Risk (Std Dev)', 'y': 'Portfolio Return', 'color': 'Sharpe Ratio'}
)

fig.add_scatter(
    x=[sharpe_risk],
    y=[sharpe_return],
    mode='markers',
    marker=dict(size=14, color='red', symbol='star'),
    name='Optimal Portfolio'
)

fig.update_layout(template='plotly_white',
                  title_x=0.5,
                  legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

fig.show()


# Student Notes: Code Explanations

This notebook builds an end-to-end portfolio optimiser. Use these notes to connect the code to the concepts.

1) Setup and data
- Import libraries, pick tickers, and download adjusted close prices for the chosen period.
- Plot prices to get a visual sense of behaviour before computing returns.

2) Returns and inputs
- Compute daily returns and annualise mean and covariance (≈252 trading days).
- These become the inputs to mean–variance optimisation.

3) Mean–variance optimisation
- Objective: minimise portfolio variance subject to weights summing to 1 and bounds.
- Solve with SLSQP and display the resulting allocation and statistics.

4) Efficient frontier
- Simulate many random portfolios to trace risk–return trade-offs and visualise the frontier.

5) Tangency (Sharpe-maximising) portfolio
- Reframe the objective to maximise Sharpe ratio and compute the tangency weights.
- Compare allocations to the variance-minimising solution and overlay on the frontier.

Tips
- If variables are undefined, re-run cells from the top; many depend on earlier outputs.
- Use `array.shape`, `df.head()`, and printed stats to sanity-check results.
- Consider transaction costs and constraints (e.g., no shorting) when interpreting weights.